<a href="https://colab.research.google.com/github/Zer0xPoint/m3u8d/blob/main/M3U8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. 安装 yt-dlp 用于下载视频
!pip install yt-dlp

# 2. 安装 Playwright
!pip install playwright

# 3. 安装 Playwright 所需的浏览器驱动（非常重要）
!playwright install

# 下载 m3u8d 可执行文件并重命名
!wget -O m3u8d https://github.com/orestonce/m3u8d/releases/download/v1.25.7/m3u8d_linux_x64_cli-v1.25.7

# 赋予文件可执行权限
!chmod +x m3u8d

# (可选) 验证一下是否成功，此命令会打印出版本号
!./m3u8d --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 17.0 MB/s eta 0:00:00
171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 52.7s171.6 MiB [] 0% 39.1s171.6 MiB [] 0% 25.6s171.6 MiB [] 0% 17.8s171.6 MiB [] 0% 10.3s171.6 MiB [] 1% 8.7s171.6 MiB [] 1% 8.0s171.6 MiB [] 2% 5.8s171.6 MiB [] 2% 5.0s171.6 MiB [] 3% 4.9s171.6 MiB [] 3% 5.1s171.6 MiB [] 4% 4.8s171.6 MiB [] 4% 5.1s171.6 MiB [] 5% 4.9s171.6 MiB [] 6% 4.4s171.6 MiB [] 6% 4.2s171.6 MiB [] 7% 3.8s171.6 MiB [] 8% 3.6s171.6 MiB [] 9% 3.5s171.6 MiB [] 10% 3.2s171.6 MiB [] 11% 3.1s171.6 MiB [] 11% 3.0s171.6 MiB [] 13% 2.8s171.6 MiB [] 14% 2.7s171.6 MiB [] 15% 2.6s171.6 MiB [] 16% 2.4s171.6 MiB [] 17% 2.3s171.6 MiB [] 18% 2.2s171.6 MiB [] 19% 2.1s171.6 MiB [] 21% 2.0s171.6 MiB [] 22% 2.0s171.6 MiB [] 22% 2.1s171.6 MiB [] 24% 1.9s171.6 MiB [] 25% 1.8s171.6 MiB [] 27% 1.7s171.6 MiB [] 28% 1.6s1

In [ ]:
#@title 👈 运行此单元格 (最终解决方案：使用正确的 m3u8d 命令)
#@markdown 请先填写下面的参数，然后点击左侧的 ▶️ 运行按钮。
# ---------------------------------#
# ----- 环境设置与兼容性修复 -----#
# ---------------------------------#
import sys
import subprocess
import nest_asyncio

# 安装并应用 nest_asyncio
try:
    nest_asyncio.apply()
    print("✅ nest_asyncio applied successfully.")
except RuntimeError: # Already applied
    print("✅ nest_asyncio was already applied.")
except ImportError:
    print("Installing nest_asyncio...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nest_asyncio"], check=True)
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio installed and applied successfully.")

import asyncio
import os
import shutil
from playwright.async_api import async_playwright, TimeoutError
import re
import shlex
from IPython.display import display, clear_output

# ---------------------------------#
# ----- 表单参数设置区域 -----#
# ---------------------------------#
#@markdown ### **1. 选择输入方式 (Choose Input Method):**
input_method = "从文件路径读取 (Read from File Path)" #@param ["直接输入URL (Direct URL Input)", "从文件路径读取 (Read from File Path)"]
#@markdown ---
#@markdown ### **2. 提供URL或文件路径 (Provide URLs or File Path):**
#@markdown **如果选择“直接输入URL”:**
video_urls_text = "https://www.mucanshipin.com/web/videoDetail/videoId/10084.html" #@param {type:"string"}
#@markdown **如果选择“从文件路径读取”:**
url_file_path = "/content/drive/MyDrive/MyDownloads/VideoLinks/轮椅小雅雅.txt" #@param {type:"string"}
#@markdown ---
#@markdown ### **3. 设置输出文件夹 (Set Output Folder):**
output_base_folder = '/content/drive/MyDrive/MyDownloads' #@param {type:"string"}
#@markdown ---
#@markdown ### **4. 设置下载线程数 (Set Download Threads):**
#@markdown 调整此参数可能有助于解决下载错误 (Adjusting this parameter may help resolve download errors).
download_threads = 3 #@param {type:"integer"}

# ---------------------------------#
# ----- 核心代码 (无需修改) -----#
# ---------------------------------#
VIDEO_TITLE_SELECTOR = 'div.video-play-box-title p.videoName'
AUTHOR_NAME_SELECTOR = 'body > section > div > div.main-video-list.ovfl > a > div.user-box.float-left > p.name'
CATEGORY_SELECTOR = 'body > section > div > div.main-video-list.ovfl > a > div.video-label.float-left'
CATEGORY_MAPPING = {"小儿麻痹症": "Polio", "截肢者": "Amputee", "轮椅": "Para"}
INVALID_CHARS_REGEX = r'[\\/*?:"<>|]'

def run_blocking_download(command):
    """在一个阻塞的同步函数中运行下载命令，并实时显示其输出。"""
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', errors='ignore')
    while True:
        output = process.stdout.readline()
        if output:
            if "%" in output:
                clear_output(wait=True)
            print(output.strip())
        if process.poll() is not None and not output:
            break
    return process.wait()

async def download_video(page_url: str, base_output_folder: str, threads: int):
    print(f"\n--- 开始处理页面 (Starting processing for page): {page_url} ---")
    browser = None
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context()
            page = await context.new_page()

            # --- 元数据抓取 ---
            await page.goto(page_url, timeout=60000)
            video_title, author_name, category_name = "untitled", "unknown", "uncategorized"
            try:
                title_element = await page.wait_for_selector(VIDEO_TITLE_SELECTOR, timeout=5000)
                video_title = await title_element.evaluate('(el) => el.firstChild.textContent.trim()')
                author_element = await page.wait_for_selector(AUTHOR_NAME_SELECTOR, timeout=5000)
                author_name = await author_element.text_content()
                category_element = await page.wait_for_selector(CATEGORY_SELECTOR, timeout=5000)
                category_text = await category_element.text_content()
                category_name = CATEGORY_MAPPING.get(category_text.strip(), category_text.strip())
                print(f"提取信息: {video_title}, {author_name}, {category_name}")
            except Exception as e:
                print(f"⚠️ 警告: 提取元数据时出错: {e}")

            # --- STAGE 1: 准备本地下载和最终路径 ---
            safe_category = re.sub(INVALID_CHARS_REGEX, "_", category_name)
            safe_author = re.sub(INVALID_CHARS_REGEX, "_", author_name)
            safe_title = re.sub(INVALID_CHARS_REGEX, "_", video_title)

            final_folder = os.path.join(base_output_folder, safe_category, safe_author)
            os.makedirs(final_folder, exist_ok=True)
            video_id = re.search(r"videoId/(\d+)", page_url).group(1) if re.search(r"videoId/(\d+)", page_url) else "unknown_id"
            final_filename = f"{safe_title}_{safe_author}_{video_id}.mp4"
            final_filepath = os.path.join(final_folder, final_filename)

            local_temp_dir = "/content/temp_downloads"
            os.makedirs(local_temp_dir, exist_ok=True)
            temp_filename_base = f"temp_video_{video_id}" # m3u8d 会自动添加 .mp4
            temp_filepath = os.path.join(local_temp_dir, f"{temp_filename_base}.mp4")

            # --- 获取下载链接 ---
            try: await page.click('#try-btn', timeout=15000)
            except TimeoutError: print("未找到播放按钮, 继续...")
            video_element = await page.wait_for_selector('video[src*="m3u8"]', timeout=30000)
            m3u8_url = await video_element.get_attribute('src')

            # 获取全部视频
            m3u8_url = re.sub(r'&end=\d+', '', m3u8_url)

            # --- 执行本地下载 ---
            print(f"步骤1: 下载到本地临时文件 {temp_filepath}")

            # *** 关键修正: 使用 'download' 模式和正确的参数 '-u', '-d', '-f' ***
            # 添加 --ThreadCount 参数控制下载线程数
            command = (
                f"./m3u8d download "
                f"-u '{m3u8_url}' "
                f"-d {shlex.quote(local_temp_dir)} "
                f"-f {shlex.quote(temp_filename_base)} "
                f"--ThreadCount {threads}" # 添加线程参数
            )
            print(f"执行命令: {command}")

            exit_code = await asyncio.get_running_loop().run_in_executor(None, run_blocking_download, command)

            # --- STAGE 2: 验证并移动文件 ---
            if exit_code == 0 and os.path.exists(temp_filepath):
                print(f"✅ 本地下载成功。")
                print(f"步骤2: 移动文件到 -> {final_filepath}")
                try:
                    shutil.move(temp_filepath, final_filepath)
                    print(f"✅ 移动成功！最终文件已保存。")
                except Exception as e:
                    print(f"❌ 移动文件时出错: {e}")
            else:
                print(f"❌ 下载到本地临时目录失败。退出码: {exit_code}。请检查上方 m3u8d 的输出日志。")

    except Exception as e:
        clear_output(wait=True)
        import traceback
        print(f"❌ 处理过程中发生未知错误: {e}")
        traceback.print_exc()
    finally:
        if browser and browser.is_connected():
            await browser.close()

async def main():
    print("正在安装/验证依赖...")
    try:
        subprocess.run(['pip', 'install', '-q', 'playwright'], check=True)
        subprocess.run(['playwright', 'install', 'chromium'], check=True, capture_output=True, text=True)
        if not os.path.exists('m3u8d'):
            print("正在下载 m3u8d...")
            subprocess.run(['wget', '-q', '-O', 'm3u8d', 'https://github.com/orestonce/m3u8d/releases/download/v1.25.7/m3u8d_linux_x64_cli-v1.25.7'], check=True)
            subprocess.run(['chmod', '+x', 'm3u8d'], check=True)
        print("✅ 所有依赖准备就绪。")
    except Exception as e:
        print(f"❌ 安装过程中发生错误: {e}")
        raise

    if os.path.exists("/content/temp_downloads"):
        shutil.rmtree("/content/temp_downloads")

    os.makedirs(output_base_folder, exist_ok=True)
    urls_to_download = []
    if "直接输入URL" in input_method:
      urls_to_download = [line.strip() for line in video_urls_text.split('\n') if line.strip()]
    elif "从文件路径读取" in input_method and os.path.exists(url_file_path):
      with open(url_file_path, 'r') as f:
        urls_to_download = [line.strip() for line in f if line.strip()]

    if not urls_to_download:
        print("⚠️ 警告: 没有提供任何有效的URL。")
        return

    for url in urls_to_download:
        await download_video(url, output_base_folder, download_threads) # 将线程参数传递给下载函数

    print("\n🎉 所有任务已处理完毕。")

asyncio.run(main())

[3/5]下载ts ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■        89.56%
